<a href="https://colab.research.google.com/github/marcio-lfo/devai/blob/main/Titanic_Ensembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Utilizando Ensembles no projeto do Titanic visando melhorar o desempenho**

In [2]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
import pandas as pd
data = pd.read_csv('/content/train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
# Verificando os tipos de dados
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
# Retiramos a coluna que queremos predizer

y = data['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [6]:
# Não vamos utilizar o data inteirno vamos retirar a coluna 'Survived'

X = data.drop('Survived', axis=1)
X.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
# Vendo quantos elementos repetidos existem nas colunas

for column in X.columns:
  print(f"{column:>12} {len(set(X[column])):4} {X[column].dtypes}")

 PassengerId  891 int64
      Pclass    3 int64
        Name  891 object
         Sex    2 object
         Age  265 float64
       SibSp    7 int64
       Parch    7 int64
      Ticket  681 object
        Fare  248 float64
       Cabin  148 object
    Embarked    4 object


Observa-se que existem várias idades
Quando a quantidade de característica é maior do que um terço da quantidade de exemplo a gente começa a perder um pouco
Para simplificação de exemplo do tutorial iremos eliminar algumas das características iremos trabalhar com outras

In [8]:
# Retirando características indesejadas

indesejadas = ['PassengerId','Name','Ticket','Cabin']
Xdrop = X.drop(indesejadas, axis=1)
Xdrop.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [9]:
# Levantando as características numéricas

Xnum = Xdrop.select_dtypes('number')
Xnum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [10]:
# Levantando as características nulas

for column in Xnum.columns:
   print(f"{column:>12} {sum(Xnum[column].isnull())}")

      Pclass 0
         Age 177
       SibSp 0
       Parch 0
        Fare 0


Observa-se que existem 177 idades nulas

In [11]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
XnumLimpo = imputer.fit_transform(Xnum)
XnumLimpo

array([[ 3.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 3.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 3.    , 28.    ,  1.    ,  2.    , 23.45  ],
       [ 1.    , 26.    ,  0.    ,  0.    , 30.    ],
       [ 3.    , 32.    ,  0.    ,  0.    ,  7.75  ]])

A estratégia padrão é a média estamos usando a mediana

imputer.fit_tranform - Pega uma entrada de dados qualquer e transforma em uma que é aceita no caso vai pegar a idade que é vazia e vai colocar a mediana

In [12]:
# Tratamento dos atributos categóricos

Xcat = Xdrop.select_dtypes('object')
Xcat.columns

Index(['Sex', 'Embarked'], dtype='object')

In [13]:
for column in Xcat.columns:
   print(f"{column:>12} {sum(Xcat[column].isnull())}")

         Sex 0
    Embarked 2


Precisamos tratar os dois casos do Embarked para o algoritimo não parar

In [14]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
XcatLimpo = imputer.fit_transform(Xcat)
XcatLimpo

array([['male', 'S'],
       ['female', 'C'],
       ['female', 'S'],
       ...,
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'Q']], dtype=object)

Outro problema, a maior parte dos classificadores que for tentar treinar eles não vão aceitar diretamente essas características categóricas.
Precisamos transformar as categorias para numéricas.
Vamos utilizar o ordinal encoder que vai colocar um número para cada situação aqui na sequência.

In [15]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
XcatHot = encoder.fit_transform(XcatLimpo)
XcatHot.toarray()

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

In [16]:
import numpy as np
Xtratado = np.c_[XnumLimpo, XcatHot.toarray()]
Xtratado.shape

(891, 10)

In [17]:
Xtratado

array([[ 3., 22.,  1., ...,  0.,  0.,  1.],
       [ 1., 38.,  1., ...,  1.,  0.,  0.],
       [ 3., 26.,  0., ...,  0.,  0.,  1.],
       ...,
       [ 3., 28.,  1., ...,  0.,  0.,  1.],
       [ 1., 26.,  0., ...,  1.,  0.,  0.],
       [ 3., 32.,  0., ...,  0.,  1.,  0.]])

In [18]:
import pandas as pd

# Verificando o que está no treino e está no teste ao mesmo tempo

train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
train.columns.isin(test.columns)

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

Observa-se que a segunda coluna não consta nas duas bases de dados

In [19]:
# Verificando qual é a coluna que tem no treino e não tem no teste

train.columns[~train.columns.isin(test.columns)]

Index(['Survived'], dtype='object')

In [20]:
# Começando um pipeline
# Começar a organizar o dataset de forma que quando chegar um dado novo cru ele vai treinar tudo de novo

from sklearn.base import BaseEstimator, TransformerMixin

# Criando uma classe para tratar os dados da base
class AtributosDesejados(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasIndesejadas_ = ['PassengerId','Name','Ticket','Cabin']
    return self

  def transform(self, X, y=None):
    return X.drop(self.colunasIndesejadas_, axis=1)

atributosDesejados = AtributosDesejados()
Xdrop = atributosDesejados.fit_transform(X)
Xdrop.columns


Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

Observe que usamos o fit_transform mas não foi implementado. O TransformerMixin já possui implementado, se tiver o transform implementado ele chama o sequência do outro.

In [21]:
# Vamos separar o que é categórico de numérico

from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasNumericas_ = X.select_dtypes('number').columns
    return self
  def transform(self, X, y=None):
    return X[self.colunasNumericas_]

atributosNumericos = AtributosNumericos()
XNum = atributosNumericos.fit_transform(Xdrop)
XNum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [22]:
# Pipeline

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipenum = Pipeline([
        ('atributos_numericos', AtributosNumericos()),
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

XnumLimpo = pipenum.fit_transform(Xdrop)
XnumLimpo

array([[ 0.82737724, -0.56573646,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.66386103,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25833709, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25833709, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.20276197, -0.4745452 , -0.47367361, -0.49237783]])

Agora ele já pega direto do drop e passa para o limpo não passa pelo xnum que faz lá dentro

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasCategoricas = X.select_dtypes(include='object').columns
    return self
  def transform(self, X, y=None):
    return X[self.colunasCategoricas]

atributosCategoricos = AtributosCategoricos()
Xcat = atributosCategoricos.fit_transform(Xdrop)
Xcat.columns

Index(['Sex', 'Embarked'], dtype='object')

In [24]:
# Pipeline

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([
        ('atributos_categoricos', AtributosCategoricos()),
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder())
    ])

XcatLimpo = pipecat.fit_transform(Xdrop)
XcatLimpo.toarray()

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

Temos dois datasets separados, um dos categórico e outro dos numérico.
Temos que junta isso tudo de novo para mandar uma matriz só para o treinar o classificador

In [25]:
# Consegue colocar os dois pipelines funcionando como um se fosse um só
from sklearn.pipeline import FeatureUnion

unecaracteristicas = FeatureUnion([
        ('pipenum', pipenum),
        ('pipecat', pipecat),
])

Xtratado = unecaracteristicas.fit_transform(Xdrop)
Xtratado.toarray()

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693,  0.66386103,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693, -0.25833709, -0.4745452 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  0.        ,
         1.        ,  0.        ]])

In [26]:
# Gerando um pipeline de todo o processamento, desde unir as características até unir as características de novo
# De forma que mandasse os dados crus e ele entregasse os dados tratados

from sklearn.pipeline import Pipeline

preproc = Pipeline([
        ('atributos_desejados', AtributosDesejados()),
        ('unecaracteristicas', unecaracteristicas)
])

Xtratado = preproc.fit_transform(X)
Xtratado.toarray()

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693,  0.66386103,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693, -0.25833709, -0.4745452 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  0.        ,
         1.        ,  0.        ]])

Até o momento só conseguimos tratar, não conseguimos classificar.
Vamos gerar um pipeline total que vai ser capaz de pegar o dado cru e entregar uma resposta.

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

pipetotal = Pipeline([
        ('preproc', preproc),
        ('arvore', DecisionTreeClassifier())
])

pipetotal.fit(X,y)
ypred = pipetotal.predict(X)
accuracy = accuracy_score(y, ypred)

print(f"Acurácia do modelo: {accuracy:.2f}")

Acurácia do modelo: 0.98


O resultado deu muito alto mais isso é uma resubstituição. Se a gente quiser verificar realmente se isso está trazendo um resultado interessante teremos que rodar uma validação cruzada.
Só vamos mandar para o teste quando souber que está legal no treinamento.
Se eu fico treinando, ajustando as coisas e testando eu vou gerar um overfitting no teste, coisas novas que chegarem não saberemos se vai ser bom ou não.

In [28]:
import numpy as np
from sklearn.model_selection import cross_validate

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([0.02633643, 0.01820922, 0.01743102, 0.01541281, 0.02168274]),
  'score_time': array([0.0066936 , 0.00836968, 0.02538896, 0.00630593, 0.01660895]),
  'test_score': array([0.72625698, 0.78651685, 0.80898876, 0.73033708, 0.80898876])},
 0.7722176887828762)

Como melhorar o resultado?
Podemos tentar ajustar as características ou o classificador. Se tentar ajustar olhando para o teste o tempo todo, vai dar problema.
Vamos gerar um exemplo de como obter esses parâmetros de forma automática.

In [29]:
from sklearn.model_selection import GridSearchCV

parametros = {
    'arvore__max_depth': [None] + list(range(1,10,2))       #  variando de 1 até 10 de 2 em 2
}

modelo = GridSearchCV(pipetotal, param_grid=parametros)
scores = cross_validate(modelo , X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([0.44970632, 0.45168018, 0.44017124, 0.45609856, 0.44784379]),
  'score_time': array([0.00436425, 0.00419927, 0.00452185, 0.00408435, 0.00452495]),
  'test_score': array([0.81564246, 0.81460674, 0.81460674, 0.78089888, 0.85393258])},
 0.8159374803841567)

Fez uma validação cruzada, aninhada para cada um dos parâmetros.

Melhorou bastante de **0.78** foi para **0.81**.

In [30]:
from sklearn.model_selection import GridSearchCV

parametros = {
    'arvore__max_depth': [None, 1, 3, 5, 7, 9],
    'preproc__unecaracteristicas__pipenum__imputer__strategy': ['mean', 'median', 'most_frequent']
}

modelo = GridSearchCV(pipetotal, param_grid=parametros)
scores = cross_validate(modelo , X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([1.41569376, 1.38296676, 1.36877418, 1.3718493 , 1.36689115]),
  'score_time': array([0.00401354, 0.00406647, 0.00393271, 0.00411248, 0.00436592]),
  'test_score': array([0.81564246, 0.81460674, 0.81460674, 0.78089888, 0.85393258])},
 0.8159374803841567)

In [31]:
modelo.fit(X,y)
modelo.best_estimator_

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('atributos_desejados', AtributosDesejados()),
                                 ('unecaracteristicas',
                                  FeatureUnion(transformer_list=[('pipenum',
                                                                  Pipeline(steps=[('atributos_numericos',
                                                                                   AtributosNumericos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('scaler',
                                                                                   StandardScaler())])),
                                                                 ('pipecat',
                                                                  Pipeline(steps=[('atributos_categoricos',
                                                                                   AtributosCategoricos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('encoder',
                                                                                   OneHotEncoder())]))]))])),
                ('arvore', DecisionTreeClassifier(max_depth=5))])

Conseguimos ver toda a sequência do nosso modelo e o que ele escolheu.
Ele escolheu o **max_depth = 5**.

No SimpleImputer a estratégia que ele escolheu foi o '**most_frequent**'.


In [32]:
# Utilizando o classificador RandomForestClassifier

from sklearn.ensemble  import RandomForestClassifier
from sklearn.metrics import accuracy_score

pipetotal = Pipeline([
        ('preproc', preproc),
        ('randomforest', RandomForestClassifier())
])

pipetotal.fit(X,y)
ypred = pipetotal.predict(X)
accuracy = accuracy_score(y, ypred)

print(f"Acurácia do modelo: {accuracy:.2f}")

Acurácia do modelo: 0.98


# **Utilizando Ensembles**

In [35]:
from sklearn.model_selection import train_test_split

Xtr, Xte, ytr, yte = train_test_split(Xtratado.toarray(), y, test_size=0.2, random_state=42)

In [36]:
# Vendo a predição, onde os classificadores divergem

from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

modelo = KNeighborsClassifier()
modelo.fit(Xtr, ytr)
knn_pred = modelo.predict(Xte)

modelo = GaussianNB()
modelo.fit(Xtr, ytr)
gnb_pred = modelo.predict(Xte)


modelo = Perceptron()
modelo.fit(Xtr, ytr)
per_pred = modelo.predict(Xte)

ypred = np.stack((knn_pred, gnb_pred, per_pred))
ypred.T

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [1, 1, 1],
       [0, 0, 1],
       [1, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [1,

In [37]:
# Classificador Perceptron

from sklearn.linear_model import Perceptron
modelo = Perceptron()
modelo.fit(Xtr, ytr)
per_pred = modelo.predict(Xte)
per_hits = per_pred == yte
sum(per_hits)/len(per_hits)

0.7318435754189944

In [38]:
# Combinando os classificadores Perceptron, GaussianNB e KNN utilizando o VotingClassifier

from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

modelo = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('gnb', GaussianNB()),
    ('per', Perceptron())
])
modelo.fit(Xtr, ytr)
vote_pred = modelo.predict(Xte)
vote_hits = vote_pred == yte
vote_hits, sum(vote_hits)/len(vote_hits)

(709    False
 439     True
 840     True
 720     True
 39      True
        ...  
 433     True
 773     True
 25      True
 84      True
 10      True
 Name: Survived, Length: 179, dtype: bool,
 0.7932960893854749)

In [39]:
# Classificador DecisionTreeClassifier

from sklearn.tree import DecisionTreeClassifier
modelo = DecisionTreeClassifier()
modelo.fit(Xtr, ytr)
dtc_pred = modelo.predict(Xte)
dtc_hits = dtc_pred == yte
sum(dtc_hits)/len(dtc_hits)

0.776536312849162

A acurácia do classificador Perceptron é de **73%**.

A Árvore de Decisão possui uma acurácia de **77%**, desempenho melhor do que a do Perceptron, o que vai melhorar o Ensemble.

In [40]:
from sklearn.ensemble import VotingClassifier
modelo = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('gnb', GaussianNB()),
    ('dtc', DecisionTreeClassifier())
])
modelo.fit(Xtr, ytr)
vote_pred = modelo.predict(Xte)
vote_hits = vote_pred == yte
vote_hits, sum(vote_hits)/len(vote_hits)

(709    False
 439     True
 840     True
 720     True
 39      True
        ...  
 433     True
 773     True
 25     False
 84      True
 10      True
 Name: Survived, Length: 179, dtype: bool,
 0.8156424581005587)

A acurácia da combinação de classificadores melhorou com a substituição do classificador **Perceptron** pelo **DecisionTreeClassifier**.

In [41]:
# Utilizando o ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier
modelo = ExtraTreesClassifier(random_state=42)
modelo.fit(Xtr, ytr)
etc_pred = modelo.predict(Xte)
etc_hits = etc_pred == yte
sum(etc_hits)/len(etc_hits)


0.8044692737430168

In [42]:
scores = cross_validate(ExtraTreesClassifier(random_state=42), Xtratado.toarray(), y)
np.mean(scores['test_score']), scores

(0.7935157868307074,
 {'fit_time': array([0.10304666, 0.09900331, 0.10201716, 0.09775186, 0.09950876]),
  'score_time': array([0.0070653 , 0.00739479, 0.00734663, 0.01011777, 0.00752354]),
  'test_score': array([0.77094972, 0.7752809 , 0.83707865, 0.78089888, 0.80337079])})

In [43]:
# Utilizando a técnica do bootstrap

from sklearn.ensemble import BaggingClassifier
modelo = BaggingClassifier(DecisionTreeClassifier(splitter='random'),
                           n_estimators=100, max_features=0.15, random_state=42)
modelo.fit(Xtr, ytr)
bag_pred = modelo.predict(Xte)
bag_hits = bag_pred == yte
sum(bag_hits)/len(bag_hits)

0.7430167597765364

In [44]:
# Utilizando um super classificador, um super Ensemble

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import StackingClassifier

voting = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('gnb', GaussianNB()),
    ('per', Perceptron())
])

modelo = StackingClassifier([
    ('voting', voting),
    ('extrat', ExtraTreesClassifier()),
    ('ranfor', RandomForestClassifier())
], cv=3, passthrough=True)

modelo.fit(Xtr, ytr)
stack_pred = modelo.predict(Xte)
stack_hits = stack_pred == yte
stack_hits, sum(stack_hits)/len(stack_hits)

(709    False
 439     True
 840     True
 720     True
 39      True
        ...  
 433     True
 773     True
 25     False
 84      True
 10      True
 Name: Survived, Length: 179, dtype: bool,
 0.8212290502793296)

Com o super classificador a acurácia passou a ser de **82%**.

In [47]:
# Utilizando o VotingClassifier e StackingClassifier no pipetotal

from scipy.sparse import issparse
from sklearn.preprocessing import FunctionTransformer

# Corrigindo problema detectado de matriz esparsa, convertendo os casos  para um array numpy denso.
def todense(X):
    if issparse(X):
        return X.toarray()
    else:
        return X

voting = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('gnb', GaussianNB()),
    ('per', Perceptron())
])

stacking = StackingClassifier([
    ('voting', voting),
    ('extrat', ExtraTreesClassifier()),
    ('ranfor', RandomForestClassifier())
], cv=3, passthrough=True)

pipetotal = Pipeline([
        ('preproc', preproc),
        ('todense', FunctionTransformer(todense, validate=False)),
        ('stacking', stacking)
])

pipetotal.fit(X,y)
ypred = pipetotal.predict(X)
accuracy = accuracy_score(y, ypred)

print(f"Acurácia do modelo: {accuracy:.2f}")

Acurácia do modelo: 0.90


In [49]:
# Validação Cruzada

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([1.35707808, 0.96555901, 0.97252345, 0.9677093 , 0.96511722]),
  'score_time': array([0.03454399, 0.03677845, 0.0337286 , 0.04620886, 0.03462911]),
  'test_score': array([0.79888268, 0.80337079, 0.83707865, 0.78651685, 0.83146067])},
 0.8114619295712762)

# **Conclusão:**

O desempenho do modelo usando o **VotingClassifier** e **StackingClassifier** (**90%**) foi melhor do que o do **GridSearchCV** (**82%**), porém, foi inferior a acurácia do classificador **RandomForestClassifier** (**98%**) submetido anteriormente no kaggle.

